# BETA_random_eqrthquake_locations_w_pyvista_vtk

Testing the random picking of subsurface eqrthquake locations (moment-tensor locations) and 3D plotting of the picks and model 

### Step 0

Load packages

In [ ]:
#load all packages
import pickle
import copy

import numpy as np
import pandas as pd
import pyvista as pv
import matplotlib.pyplot as plt 

from sys import argv

from matplotlib.colors import Normalize
from pyaspect.model.gridmod3d import gridmod3d as gm
from pyaspect.model.bbox import bbox as bb
from pyaspect.model.gm3d_utils import compress_gm3d_to_file
from pyaspect.model.gm3d_utils import decompress_gm3d_from_file
from pyaspect.moment_tensor import MomentTensor
from pyaspect.specfemio.headers import StationHeader
from pyaspect.specfemio.headers import SolutionHeader
from pyaspect.specfemio.headers import CMTSolutionHeader
from pyaspect.specfemio.headers import ForceSolutionHeader
from pyaspect.specfemio.write import write_cmtsolution
from pyaspect.specfemio.write import write_forcesolution
from pyaspect.specfemio.write import write_stations
from pyaspect.specfemio.read import read_stations
from pyaspect.specfemio.read import read_solution
from pyaspect.specfemio.read import read_cmtsolution
from pyaspect.specfemio.read import read_forcesolution
from pyaspect.specfemio.utils import get_xyz_coords_from_station_list
from pyaspect.specfemio.utils import get_xyz_coords_from_station_list_except
from pyaspect.specfemio.utils import *

### Step 1 

Extract the ndarray of the subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
data_in_dir  = 'data/output/'
data_out_dir = data_in_dir
!ls {data_in_dir} 

### Step 6 

Decompress the ndarray of the sliced, subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
# set filename then used it to decompress model
ifqn = f'{data_out_dir}/vsliced_subsmp_smth_nam_2017_vp_vs_rho_Q_model_dx100_dy100_dz100_maxdepth5850_sig250.npz'
vslice_gm3d, other_pars = decompress_gm3d_from_file(ifqn)

print()
print('decompressed gridded model\n:',vslice_gm3d) 
print()
print('other parameters:\n',other_pars)
print()

# WARNING: this will unpack all other_pars, if you overwrite a variable of the samename as val(key), then you 
#          may not notice, and this may cause large headaches.  I use it because I am aware of it.
'''
for key in other_pars:
    locals()[key] = other_pars[key]  #this is more advanced python than I think is reasonable for most 
sig_meters = sig
''';

# another way to get these varibles is just use the accessor functions for the gridmod3d.  We need them later.
xmin = other_pars['xmin']
dx   = other_pars['dx']
nx   = other_pars['nx']
ymin = other_pars['ymin']
dy   = other_pars['dy']
ny   = other_pars['ny']
zmin = other_pars['zmin']
dz   = other_pars['dz']
nz   = other_pars['nz']
sig_meters = other_pars['sig']  # this variable is used later
print('sig_meters:',sig_meters)

In [ ]:
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape + 1 because we want to inject our values on
#   the CELL data
nam_dims = list(vslice_gm3d.get_npoints())
nam_origin = list(vslice_gm3d.get_gorigin())
nam_origin[2] *= -1
nam_origin = tuple(nam_origin)
nam_spacing = list(vslice_gm3d.get_deltas())
nam_spacing[2] *=-1
nam_spacing = tuple(nam_spacing)
print('nam_dims:',nam_dims)
print('nam_origin:',nam_origin)
print('nam_spacing:',nam_spacing)

# Edit the spatial reference
grid.dimensions = np.array(nam_dims) + 1
grid.origin = nam_origin  # The bottom left corner of the data set
grid.spacing = nam_spacing  # These are the cell sizes along each axis
nam_pvalues = vslice_gm3d.getNPArray()[0]
print('pvalues.shape:',nam_pvalues.shape)

# Add the data values to the cell data
grid.cell_arrays["values"] = nam_pvalues.flatten(order="F")  # Flatten the array!

# Now plot the grid!
cmap = plt.cm.jet
#grid.plot(show_edges=True,cmap=cmap)
grid.plot(cmap=cmap,opacity=1.0)
print('dir:\n',dir(grid))


In [ ]:
slices = grid.slice_orthogonal()

#slices.plot(show_edges=True,cmap=cmap)
slices.plot(cmap=cmap)

In [ ]:
print('vslice_gm32:\n',vslice_gm3d)
coords = vslice_gm3d.getGlobalCoordsPointsXYZ()
coords[:,2] = -coords[:,2]
print(coords.shape)
print('norm:\n',coords)

xc = vslice_gm3d.getLocalCoordsPointsX()
yc = vslice_gm3d.getLocalCoordsPointsY()
zc = vslice_gm3d.getLocalCoordsPointsZ()

thing = np.meshgrid(xc,yc,zc)
print('thing0.shape:',thing[0].shape)
print('thing1.shape:',thing[1].shape)
print('thing2.shape:',thing[2].shape)
print('thing0:\n',thing[0])
print('thing1:\n',thing[1])
print('thing2:\n',thing[2])

'''
print(coords.shape)
coords = coords.transpose()
#coords[0,:] += np.arange(4)*100000
#coords[1,:] -= np.arange(4)*100000
print('transp:\n',coords)
sort_coords = coords [ :, coords[2].argsort()]
coords = sort_coords
print('sort:\n',coords)
coords = coords.transpose()
print('norm:\n',coords)
coords = coords[::4,:]
''';
def skip_by_index(arr,i,skip):
    #print('arr norm-shape:\n',arr.shape)
    arr = arr.transpose()
    print('arr trans:\n',arr)
    sort_arr = arr[ :, arr[i].argsort()]
    arr = sort_arr
    arr = arr.transpose()
    arr = arr[::skip,:]
    return arr

def get_unique_xyz_skip(arr,skip):
    ux = np.unique(arr.T[0,:])
    print('nx:',len(np.unique(arr.T[0,:])))
    uy = np.unique(arr.T[1,:])
    print('ny:',len(np.unique(arr.T[1,:])))
    uz = np.unique(arr.T[2,:])
    print('nz:',len(np.unique(arr.T[2,:])))
    
    ux = ux[::skip]
    uy = uy[::skip]
    uz = uz[::skip]
    
    return np.vstack(np.meshgrid(ux,uy,uz)).reshape(3,-1).T
   
coords = get_unique_xyz_skip(coords,8)



#coords = skip_by_index(coords,2,8)
#coords = skip_by_index(coords,1,8)
#coords = skip_by_index(coords,0,8)

print('coords after sort:\n',coords)
print('coords after shape:\n',coords.shape)


'''
coords = coords[::4,:]
print()
print(coords.shape)
print(coords)
coords[:,2] = -coords[:,2]
print(coords)
coords = coords.transpose()
print('transp:\n',coords)
sort_coords = coords [ :, coords[2].argsort()]
coords = sort_coords
coords = coords[:,::4].transpose()
sort_coords = coords [ :, coords[1].argsort()]
coords = sort_coords
coords = coords[:,::4].transpose()
print('sort:\n',coords)
''';

In [ ]:
pv_points = pv.wrap(coords)
p = pv.Plotter()
#p.add_mesh(slices,cmap=cmap)
p.add_mesh(grid,cmap=cmap,opacity=0.85)
p.add_mesh(pv_points, render_points_as_spheres=True, point_size=10,opacity=0.85)
p.show()

In [ ]:
coords = vslice_gm3d.getGlobalCoordsPointsXYZ()
coords[:,2] = -coords[:,2]

xc = np.unique(coords.T[0,:])
yc = np.unique(coords.T[1,:])
zc = np.unique(coords.T[2,:])

#print('nx:\n',len(xc))
#print('x:\n',xc)


n_rx = 5
n_ry = 5
n_rz = 5
n_rand_p = 1000

lrx = np.min(xc)
lry = np.min(yc)
lrz = -4300.0

hrx = np.max(xc)
hry = np.max(yc)
hrz = -3000.0

srx = hrx - lrx
sry = hry - lry
srz = hrz - lrz

r_xyz_list = []
#for i in range(n_rx*n_ry*n_rz):
for i in range(n_rand_p):
    rx = lrx + srx*np.random.rand()
    ry = lry + sry*np.random.rand()
    rz = lrz + srz*np.random.rand()
    r_xyz_list.append([rx,ry,rz])
    
r_xyz = np.array(r_xyz_list)
    

#rx = lrx + srx*np.random.rand(n_rx)
#ry = lry + sry*np.random.rand(n_ry)
#rz = lrz + srz*np.random.rand(n_rz)

#print('rx:\n',rx)
#print('ry:\n',ry)
#print('rz:\n',rz)

#r_xyz = np.vstack(np.meshgrid(rx,ry,rz)).reshape(3,-1).T
print('r_xyz:\n',r_xyz)


In [ ]:
pv_rpoints = pv.wrap(r_xyz)
p = pv.Plotter()
#p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.75)
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

In [ ]:
print(len(r_xyz))

In [ ]:
!pwd

In [ ]:
#!mkdir {data_out_dir}/tmp
!ls {data_out_dir}

In [ ]:

if 'l_grp_stations' in locals() or 'l_grp_stations' in globals():
    print('deleting')
    del l_grp_stations
    
l_stations = []

for i in range(len(r_xyz)):
    
    name = 't' + str(i).zfill(len(str(len(r_xyz))))
    new_s = StationHeader(name=name,
                          network='NL',
                          lat_yc=r_xyz[i,1],
                          lon_xc=r_xyz[i,0],
                          elevation=0.0,
                          burial=r_xyz[i,2],
                          trid=i)
    l_stations.append(new_s)
print('len(l_stats):',len(l_stations))
                                           
l_grp_stations = make_grouped_cross_station_headers(stations=l_stations,delta=250.0)
#l_grp_stations += [l_grp_stations[0]]
t_grp_stations = sorted(copy.deepcopy(flatten_grouped_headers(l_grp_stations)))
print('len(t_grp):',len(t_grp_stations))
print('len(t_grp[0]):',len([t_grp_stations[0]]))
t_grp_stations += [t_grp_stations[0]]
print('len(t_grp):',len(t_grp_stations))
s_grp_stations = sorted(copy.deepcopy(flatten_grouped_headers_unique(l_grp_stations)))
print('len(s_grp):',len(s_grp_stations))
print('type(stat):',type(s_grp_stations))
ts = copy.deepcopy(s_grp_stations[-1])
ts.trid = -2
ts.name = 'dummy'
ts['goobid'] = 13
print('ts:\n',ts)
s_grp_stations += [ts]
for i in range(10):
    s_grp_stations += [ts]

In [ ]:
s_grp_stations = list(sorted(set(s_grp_stations)))
print('len:',len(s_grp_stations))

In [ ]:
print(s_grp_stations[0])
print(s_grp_stations[5])

In [ ]:
s_grp_stations = sorted(s_grp_stations)
print('dummy.hash:31880328642150367')
#for s in sorted(l_grp_stations):
    #print(s.name)
#write_stations(data_out_dir + '/tmp',l_stations)
#write_stations(data_out_dir + '/tmp',l_grp_stations)
#write_stations(data_out_dir + '/tmp',s_grp_stations)
write_stations(data_out_dir + '/tmp',s_grp_stations)
write_stations(data_out_dir + '/tmp',l_grp_stations,fname='STATIONS_GROUPED')

In [ ]:
!ls -ltrh {data_out_dir + '/tmp'} 

In [ ]:
!head {data_out_dir}/tmp/STATIONS

In [ ]:
!tail {data_out_dir}/tmp/STATIONS

In [ ]:
!head {data_out_dir}/tmp/STATIONS_GROUPED

In [ ]:
!tail {data_out_dir}/tmp/STATIONS_GROUPED

In [ ]:
!diff {data_out_dir}/tmp/STATIONS {data_out_dir}/tmp/STATIONS_GROUPED

In [ ]:
fqp = data_out_dir + '/tmp'
rw_stations = sorted(read_stations(fqp))

for s in sorted(rw_stations):
    print(f'{s.name}_{s.trid}_{s.gid}_{s.sid}')

In [ ]:
import os

path = os.path.relpath(data_out_dir + '///', start=os.curdir)
newp = os.path.join(path, 'tmp', 'STATIONS')

print(newp)


In [ ]:
!ls /Users/seismac/Documents/Work/Bench/ForkGnam/pyaspect/notebooks/Full_Workflow/FWI_Workflow_for_Groningen


In [ ]:
import datetime

                 
print()
print('--------------------Test CMTSolution--------------------------------------------------')
print()

test_cmt_h = CMTSolutionHeader(date=datetime.datetime.now(),
                               ename='test_event',
                               tshift=0.0,
                               hdur=0.0,
                               lat_yc=2.71,
                               lon_xc=3.14,
                               depth=-5432,
                               mt=MomentTensor(mw=3.5,strike=120,dip=40,rake=15),
                               eid=7,
                               sid=77)

print(test_cmt_h)
print()
test_cmt_h.mrr = -1 
print(test_cmt_h)
print()
#tmt = test_cmt_h.mt
#print('mt:',tmt)
print(test_cmt_h)
print()

other_cmt_h = copy.deepcopy(test_cmt_h)
other_cmt_h['goobid'] = -12
#other_cmt_h.depth = 12

test_set = set([other_cmt_h,test_cmt_h,test_cmt_h])
print('Set test:', test_set)
print()
print('let(set):', len(test_set))




test_force_h = ForceSolutionHeader(date=datetime.datetime.now(),
                                   tshift=0.0,
                                   f0=0.0,
                                   lat_yc=2.71,
                                   lon_xc=3.17,
                                   depth=13,
                                   factor_fs=1,
                                   comp_src_EX=1,
                                   comp_src_NY=0,
                                   comp_src_Zup=0,
                                   eid=7,
                                   sid=77)

print()
print('--------------------Test FroceSolution------------------------------------------------')
print()
print(test_force_h)
print()
other_force_h = copy.deepcopy(test_force_h)
other_force_h['goobid'] = -12
other_force_h.comp_src_EX = 0
other_force_h.comp_src_Zup = 1
print(other_force_h)

test_set = set([other_force_h,test_force_h,test_force_h])
print('Set test:', test_set)
print()
print('len(set):', len(test_set))

In [ ]:
write_cmtsolution(data_out_dir + '/tmp',test_cmt_h)
write_forcesolution(data_out_dir + '/tmp',test_force_h)

In [ ]:
!ls -ltrh {data_out_dir + '/tmp'} 

In [ ]:
!cat {data_out_dir}/tmp/CMTSOLUTION

In [ ]:
!cat {data_out_dir}/tmp/FORCESOLUTION

In [ ]:
fqp = f'{data_out_dir}/tmp'
read_gen_cmt = read_solution(fqp,'CMTSOLUTION')
read_gen_fs  = read_solution(fqp,'FORCESOLUTION')
read_cmt = read_cmtsolution(fqp,fname='CMTSOLUTION')
read_fs  = read_forcesolution(fqp,fname='FORCESOLUTION')
    
print('gen cmt:\n',read_gen_cmt)
print()
print('gen fs:\n',read_gen_fs)
print()
print('cmt:\n',read_cmt)
print()
print('fs:\n',read_fs)
print()

In [ ]:
all_g_xyz = get_xyz_coords_from_station_list(rw_stations)
exc_g_xyz = get_xyz_coords_from_station_list_except(rw_stations,key='gid',val=0)

In [ ]:
pv_all_points = pv.wrap(all_g_xyz)
p = pv.Plotter()
p.add_mesh(grid,cmap=cmap,opacity=0.75)
p.add_mesh(pv_all_points, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

In [ ]:
pv_exc_points = pv.wrap(exc_g_xyz)
p = pv.Plotter()
p.add_mesh(grid,cmap=cmap,opacity=0.75)
p.add_mesh(pv_exc_points, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

In [ ]:
pv_rpoints = pv.wrap(r_xyz)
p = pv.Plotter()
p.add_mesh(grid,cmap=cmap,opacity=0.75)
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

In [ ]:
class StationTrace(object):
    
    def __init__(self, header=None, data=None, start_t=0, dt=1):
        
        if not isinstance(header,StationHeader):
            raise Exception('arg: \'header\' must be of type StationHeader')
            
        self.header  = copy.deepcopy(header)
        self.data    = np.array(data)
        self.start_t = start_t
        self.dt      = dt
        
        
    def __getitem__(self, islice):
        print(dir(islice))
        return self.data[islice]
    
    def __setitem__(self, islice, value):
        self.data[islice] = value
        
    def __str__(self):
        out_str  = f'Station Header:\n{self.header}\n'
        out_str += f'Data:\n{self.data}'
        return out_str
    
    def __repr__(self):
        out_str  = f'Station Header:\n{self.header.__repr__()}\n'
        out_str += f'Data:\n{self.data.__repr__()}'
        return out_str
    
    def time_slice(self,*args):
        tslice = slice(*args)
        
        if tslice.start == None and tslice.step == None:
            print('if')
            return self.start_t + tslice.stop*self.dt
        
        elif  tslice.step == None:
            print('elif')
            if tslice.stop <= tslice.start:
                raise Exception('stop index must be greater than start index')
                
            return self.start_t + np.arange(tslice.start,tslice.stop)*self.dt
            
        else:
            print('else')
            if tslice.stop <= tslice.start:
                raise Exception('stop index must be greater than start index')
                
            return self.start_t + np.arange(tslice.start,tslice.stop,tslice.step)*self.dt
        
        
    def data_and_time(self,*args):
        islice = slice(*args)
        return self[islice], self.time_slice(islice)
    '''
    '''
        
        
        
        
        
        
    
class Record(object):
    
    def __init__(self, solution_h=None, stations_h=None, rid=0, iter_id=0):
        
        if not isinstance(solution_h,SolutionHeader):
            raise Exception('arg: \'solution_h\' must be of type SolutionHeader')
            
        check_all = all(isinstance(s,StationHeader) for s in stations_h)
        if not check_all:
            raise Exception('elements in arg: \'stations_h\' must be of type StationHeader')
            
        check_all = all(s.sid == solution_h.sid for s in stations_h)
        if not check_all:
            raise Exception('sid of each element in arg: \'stations_h\' must match solution_h.sid')
            
        self.solution_header = copy.deepcopy(solution_h)
        self.station_headers = copy.deepcopy(stations_h)
        
        self.rid     = rid
        self.iter_id = iter_id
        
        self.df = pd.DataFrame.from_records(self.station_headers, index=['sid','trid','gid'])
        
        self.traces = []
        
        self.added_header_words = []
        
        
    def __str__(self):
        out_str  = f'Solution Header:\n{self.solution_header}\n'
        out_str += f'Station Headers:\n {self.df}'
        return out_str
    
    def __repr__(self):
        out_str  = f'Solution Header:\n{self.solution_header.__repr__()}\n'
        out_str += f'Station Headers:\n {self.df.__repr__()}'
        return out_str
    
    
    def _update_df(self):
        df_index = copy.deepcopy(self.df.index)
        del self.df
        self.df = pd.DataFrame.from_records(self.station_headers, index=df_index)
    
    def add_header_word_to_stations(self, func=None):
        
        if not callable(func):
            raise Exception('arg: \'func\' must be a function')
            
        key = func(self)
        self.added_header_words.append(key)
            
        self._update_df()
            
        
        
        
#df = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list("ABCD"))
#df = pd.DataFrame.from_records(l_grp_stations)
#df = pd.DataFrame.from_records(flatten_grouped_headers(l_grp_stations),index=['sid','trid','gid'])
#headers = [{'A': 'apple', 'B': 0, 'C': 3, 'D': 3}, {'A': 'banana', 'B': 9, 'C': 3, 'D': 5}, {'A': 'orange', 'B': 4, 'C': 7, 'D': 6}]
#df = pd.DataFrame(headers)
#print('df:\n',df[0:18])

df_force_h = test_force_h
df_force_h.sid = 0
print('df_force_h.sid:',df_force_h.sid)
print()
t_record = Record(solution_h=df_force_h,stations_h=flatten_grouped_headers(l_grp_stations))

print(t_record)
print()

def offset_func(self):
    for s in self.station_headers:
        x_sqrd = (self.solution_header.lon_xc - s.lon_xc)**2
        y_sqrd = (self.solution_header.lat_yc - s.lat_yc)**2
        offset = np.sqrt(x_sqrd + y_sqrd)
        s['offset'] = offset
        return 'offset'
        
    
t_record.add_header_word_to_stations(offset_func)

print(t_record)
print()
print(t_record.added_header_words)
print()

t_st = StationTrace(s_grp_stations[0],np.arange(2000), start_t=0, dt=0.25)

print(t_st)
print()

t_st[:3] = -1*np.ones((3))
print(t_st)
print()
#t_st.time_slice(0,10,2)
myslice = t_st.time_slice(2,10,2)
print(f'myslice:\n{myslice}')

t_st[4:20:2]

In [ ]:
def silly_func(x):
    print('Hahwoe!')

silly_func = 'Hahwoe!'
    
print(callable(silly_func))

In [ ]:
mys = slice(0,10,2)
print('dir:',dir(mys))
print()
print('dir(index):',dir(mys.indices))
print()
print('start:',mys.start)
print('stop :',mys.stop)
print('step :',mys.step)